In [1]:
import pandas as pd
import pathlib
import numpy as np
from typing import List
from pathlib import Path
from pandas import DataFrame

# 数据读取

In [2]:
data_source = Path('data').joinpath('data.csv')
data_source

PosixPath('data/data.csv')

In [3]:
df = pd.read_csv(data_source)
df.head()

,numeric_1,numeric_2,numeric_3,客户类型,国际,是否私行客户,date
0,306.0,414.0,388.0,机构客户,中国,1.0,2022-10-14 22:07:22
1,309.0,145.0,505.0,机构客户,俄罗斯,0.0,2021-12-07 15:18:45
2,840.0,929.0,797.0,机构客户,法国,0.0,2021-06-03 11:22:03
3,908.0,502.0,194.0,私人客户,德国,1.0,2023-10-12 15:46:56
4,319.0,654.0,NaN,私人客户,巴西,0.0,2021-03-24 06:43:11


# 数据预处理

### 变量类型判断

In [4]:
# 根据unique值数量初筛连续变量和分类变量,阈值可调整
target_col = '是否私行客户'
_cols = set(df.columns.to_list())
_cols.remove(target_col)

num_col_threshold = 20 # 超过20个不同变量则为Numerical
num_cols = []
cate_cols = []
for col in _cols:
    unique_vals = list(df[col].unique())
    if len(unique_vals) > num_col_threshold:
        num_cols.append(col)
    else:
        cate_cols.append(col)

In [5]:
num_cols

['numeric_2', 'date', 'numeric_1', 'numeric_3']

In [6]:
cate_cols

['国际', '客户类型']

### 变量类型校准

In [7]:
num_cols.remove('date')

## 数据处理

### 删除空值比例高于X%的列

In [8]:
def remove_missing_cols(df:DataFrame, threshold:float = 0.8):
    df = df.copy()
    _drop_cols = []
    for col in df.columns: 
        percent_missing = df[col].isnull().sum() / len(df)
        if percent_missing > threshold:
            _drop_cols.append(col)
    return df.drop(_drop_cols, axis=1)

In [9]:
df_processed = remove_missing_cols(df,0.8)
df_processed.head()

,numeric_1,numeric_2,numeric_3,客户类型,国际,是否私行客户,date
0,306.0,414.0,388.0,机构客户,中国,1.0,2022-10-14 22:07:22
1,309.0,145.0,505.0,机构客户,俄罗斯,0.0,2021-12-07 15:18:45
2,840.0,929.0,797.0,机构客户,法国,0.0,2021-06-03 11:22:03
3,908.0,502.0,194.0,私人客户,德国,1.0,2023-10-12 15:46:56
4,319.0,654.0,NaN,私人客户,巴西,0.0,2021-03-24 06:43:11


### 缺失值统计

In [10]:
def null_statistic(df):
    return df.isnull().sum().to_csv('null_statistic.csv')

In [11]:
null_statistic(df_processed)

### 缺失值填充

In [12]:
def null_filler(series: pd.Series, option: str, quantile_val:float = None) -> pd.Series :
    if pd.api.types.is_string_dtype(series):
        return series.mode()
    
    if option == 'mean':
        return series.mean()
    
    if option == 'median':
        return series.median()
    
    if option == 'mode':
        return series.mode()
    
    if option == 'quantile':
        return series.quantile(quantile_val)
    
    
    return 0
    

def fill_null(df:DataFrame, options:str | List[str], quantile_val:float=None) -> DataFrame:
    """Fill dataset with different options
    options can be median,mean,mode, quantile
    """
    _df = df.copy()
    if type(options) == str:
        options = [options] * len(_df.columns)
        
    for col,option in zip(_df.columns,options):
        _val = null_filler(df[col],option, quantile_val)
        _df[col] = _df[col].fillna(_val)
        
    return _df

In [13]:
df_processed = fill_null(df_processed,'quantile',0.25)
df_processed.head()

,numeric_1,numeric_2,numeric_3,客户类型,国际,是否私行客户,date
0,306.0,414.0,388.00,机构客户,中国,1.0,2022-10-14 22:07:22
1,309.0,145.0,505.00,机构客户,俄罗斯,0.0,2021-12-07 15:18:45
2,840.0,929.0,797.00,机构客户,法国,0.0,2021-06-03 11:22:03
3,908.0,502.0,194.00,私人客户,德国,1.0,2023-10-12 15:46:56
4,319.0,654.0,344.25,私人客户,巴西,0.0,2021-03-24 06:43:11


### 处理异常值

In [30]:
df_describe = df_processed[num_cols]
df_describe = df_processed.describe()
df_describe

,numeric_1,numeric_2,numeric_3,是否私行客户,国际_中国,国际_俄罗斯,国际_加拿大,国际_印度,国际_巴西,国际_德国,国际_日本,国际_法国,国际_澳大利亚,国际_美国,客户类型_机构客户,客户类型_私人客户
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,742.915000,782.486000,759.647500,0.457000,0.079000,0.112000,0.090000,0.10100,0.086000,0.107000,0.104000,0.093000,0.095000,0.089000,0.477000,0.482000
std,1143.323482,1258.131793,1184.851364,0.498397,0.269874,0.315524,0.286325,0.30148,0.280504,0.309268,0.305413,0.290578,0.293362,0.284886,0.499721,0.499926
min,101.000000,100.000000,101.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,333.000000,343.000000,344.250000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,544.000000,553.500000,564.500000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,780.250000,809.500000,805.250000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,9860.000000,9950.000000,9940.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# for col_name, col_info in df_describe.to_dict().items():
    # pass
    # df_processed = df_processed[df_processed[col_name] < col_info['75%']]
    # df_processed = df_processed[df_processed[col_name] > col_info['25%']]
df_processed.head()

### Dummy variables

In [15]:
def process_dummies(df:DataFrame,dummy_cols:List) -> DataFrame:
    """Convert categorical variable into dummies"""
    return pd.get_dummies(df,columns=dummy_cols)

In [16]:
df_processed = process_dummies(df_processed,cate_cols)
df_processed.head()

,numeric_1,numeric_2,numeric_3,是否私行客户,date,国际_中国,国际_俄罗斯,国际_加拿大,国际_印度,国际_巴西,国际_德国,国际_日本,国际_法国,国际_澳大利亚,国际_美国,客户类型_机构客户,客户类型_私人客户
0,306.0,414.0,388.00,1.0,2022-10-14 22:07:22,1,0,0,0,0,0,0,0,0,0,1,0
1,309.0,145.0,505.00,0.0,2021-12-07 15:18:45,0,1,0,0,0,0,0,0,0,0,1,0
2,840.0,929.0,797.00,0.0,2021-06-03 11:22:03,0,0,0,0,0,0,0,1,0,0,1,0
3,908.0,502.0,194.00,1.0,2023-10-12 15:46:56,0,0,0,0,0,1,0,0,0,0,0,1
4,319.0,654.0,344.25,0.0,2021-03-24 06:43:11,0,0,0,0,1,0,0,0,0,0,0,1


### Add quntile info

In [17]:
def add_quantiles(df:DataFrame, numeric_cols:List, num_qutiles:int) -> DataFrame:
    """Add quntile info for selecte numeric columns"""
    _df = df.copy()
    for col in numeric_cols:
        _df[f'q_{col}'] = pd.qcut(_df[col],num_qutiles,labels=False)
    return _df

In [18]:
add_quantiles(df_processed,num_cols,10).head()

,numeric_1,numeric_2,numeric_3,是否私行客户,date,国际_中国,国际_俄罗斯,国际_加拿大,国际_印度,国际_巴西,国际_德国,国际_日本,国际_法国,国际_澳大利亚,国际_美国,客户类型_机构客户,客户类型_私人客户,q_numeric_2,q_numeric_1,q_numeric_3
0,306.0,414.0,388.00,1.0,2022-10-14 22:07:22,1,0,0,0,0,0,0,0,0,0,1,0,3,2,3
1,309.0,145.0,505.00,0.0,2021-12-07 15:18:45,0,1,0,0,0,0,0,0,0,0,1,0,0,2,4
2,840.0,929.0,797.00,0.0,2021-06-03 11:22:03,0,0,0,0,0,0,0,1,0,0,1,0,8,8,7
3,908.0,502.0,194.00,1.0,2023-10-12 15:46:56,0,0,0,0,0,1,0,0,0,0,0,1,4,8,0
4,319.0,654.0,344.25,0.0,2021-03-24 06:43:11,0,0,0,0,1,0,0,0,0,0,0,1,6,2,2


## Data Analysis

In [19]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   numeric_1  1000 non-null   float64
 1   numeric_2  1000 non-null   float64
 2   numeric_3  1000 non-null   float64
 3   是否私行客户     1000 non-null   float64
 4   date       1000 non-null   object 
 5   国际_中国      1000 non-null   uint8  
 6   国际_俄罗斯     1000 non-null   uint8  
 7   国际_加拿大     1000 non-null   uint8  
 8   国际_印度      1000 non-null   uint8  
 9   国际_巴西      1000 non-null   uint8  
 10  国际_德国      1000 non-null   uint8  
 11  国际_日本      1000 non-null   uint8  
 12  国际_法国      1000 non-null   uint8  
 13  国际_澳大利亚    1000 non-null   uint8  
 14  国际_美国      1000 non-null   uint8  
 15  客户类型_机构客户  1000 non-null   uint8  
 16  客户类型_私人客户  1000 non-null   uint8  
dtypes: float64(4), object(1), uint8(12)
memory usage: 50.9+ KB


## Compose Dataset

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split

In [21]:
df_data = df_processed.copy()
target_col = '是否私行客户' # Class column
feature_cols = df_data.columns.to_list() # Features
feature_cols.remove(target_col)
feature_cols.remove('date')

In [22]:
df_data.head()

,numeric_1,numeric_2,numeric_3,是否私行客户,date,国际_中国,国际_俄罗斯,国际_加拿大,国际_印度,国际_巴西,国际_德国,国际_日本,国际_法国,国际_澳大利亚,国际_美国,客户类型_机构客户,客户类型_私人客户
0,306.0,414.0,388.00,1.0,2022-10-14 22:07:22,1,0,0,0,0,0,0,0,0,0,1,0
1,309.0,145.0,505.00,0.0,2021-12-07 15:18:45,0,1,0,0,0,0,0,0,0,0,1,0
2,840.0,929.0,797.00,0.0,2021-06-03 11:22:03,0,0,0,0,0,0,0,1,0,0,1,0
3,908.0,502.0,194.00,1.0,2023-10-12 15:46:56,0,0,0,0,0,1,0,0,0,0,0,1
4,319.0,654.0,344.25,0.0,2021-03-24 06:43:11,0,0,0,0,1,0,0,0,0,0,0,1


### Correlation Coefficient

In [32]:
df_data.corrwith(df[target_col],method='pearson') # Correlation coefficient for eacth feature column

/var/folders/ld/d3kwjb6948b9601kvb31ykpr0000gp/T/ipykernel_19260/2911165685.py:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_data.corrwith(df[target_col],method='pearson') # Correlation coefficient for eacth feature column


numeric_1   -0.024873
numeric_2    0.019487
numeric_3    0.052319
是否私行客户       1.000000
国际_中国        0.006608
国际_俄罗斯      -0.010549
国际_加拿大      -0.032766
国际_印度       -0.040126
国际_巴西       -0.048240
国际_德国        0.043183
国际_日本        0.040160
国际_法国       -0.027720
国际_澳大利亚      0.040579
国际_美国        0.022347
客户类型_机构客户    0.040671
客户类型_私人客户   -0.045570
dtype: float64

### WOE & IV Value

## Build models

In [24]:
df_data = df_data.dropna()
X_train, X_test, y_train, y_test = train_test_split(df_data[feature_cols], df_data[target_col], test_size=0.33, random_state=42)

### Normalization

In [25]:
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

### Train models

In [26]:
from sklearn.linear_model import LogisticRegression
reg_log = LogisticRegression()
reg_log.fit(X_train, y_train)
predictions = reg_log.predict(X_test)

In [27]:
models = {}

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression()

# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
models['Naive Bayes'] = GaussianNB()

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy, precision, recall, f1, TN, FP, FN, TP = {}, {}, {}, {}, {}, {}, {}, {}

for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train, y_train)
    
    # Make predictions
    predictions = models[key].predict(X_test)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)
    f1[key] = f1_score(predictions,y_test)
    TN[key], FP[key], FN[key], TP[key] = confusion_matrix(y_test, predictions).ravel()

/Users/junxiaoguo/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [29]:
import pandas as pd

df_model = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall','F1'])
df_model['Accuracy'] = accuracy.values()
df_model['Precision'] = precision.values()
df_model['Recall'] = recall.values()
df_model['F1'] = f1.values()
df_model['TN'] = TN.values()
df_model['FP'] = FP.values()
df_model['FN'] = FN.values()
df_model['TP'] = TP.values()


df_model

,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
Logistic Regression,0.539394,0.346939,0.476636,0.401575,127,56,96,51
Support Vector Machines,0.536364,0.346939,0.472222,0.400000,126,57,96,51
Decision Trees,0.503030,0.530612,0.450867,0.487500,88,95,69,78
Random Forest,0.527273,0.428571,0.466667,0.446809,111,72,84,63
Naive Bayes,0.506061,0.639456,0.460784,0.535613,73,110,53,94
